In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_pir_current
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) AS
SELECT      
    FARM_FINGERPRINT(CONCAT(
        IFNULL(CAST(articleno AS STRING), ""),
        IFNULL(CAST(vendor AS STRING), "")
    )) hash_id,
    pir,
    articleno AS product_id,
    category,
    vendor AS vendor_id,
    subrange AS subrange_id,
    delete_indicator,
    base_unit_of_measure AS base_unit,
    order_unit,
    country_of_origin,
    return_agreement,
    regular_vendor,
    IFNULL(availbleto, '9999-12-31') AS available_to,
    IF(delete_indicator IS NULL AND DATE(CURRENT_TIMESTAMP()+INTERVAL 7 HOURS) < IFNULL(availbleto, '9999-12-31'), 'ACTIVE', 'INACTIVE') AS status           
FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmpo09
WHERE end_date = '2400-01-01'
QUALIFY ROW_NUMBER() OVER (PARTITION BY vendor, articleno ORDER BY proc_date DESC) = 1
""")